In [49]:
from random import randint
import random
import timeit
from copy import copy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [194]:
"""
This file contains the `Board` class, which implements the rules for the
game Isolation as described in lecture, modified so that the players move
like knights in chess rather than queens.

You MAY use and modify this class, however ALL function signatures must
remain compatible with the defaults provided, and none of your changes will
be available to project reviewers.
"""


TIME_LIMIT_MILLIS = 150


class Board(object):
    """Implement a model for the game Isolation assuming each player moves like
    a knight in chess.

    Parameters
    ----------
    player_1 : object
        An object with a get_move() function. This is the only function
        directly called by the Board class for each player.

    player_2 : object
        An object with a get_move() function. This is the only function
        directly called by the Board class for each player.

    width : int (optional)
        The number of columns that the board should have.

    height : int (optional)
        The number of rows that the board should have.
    """
    BLANK = 0
    NOT_MOVED = None

    def __init__(self, player_1, player_2, width=7, height=7):
        self.width = width
        self.height = height
        self.move_count = 0
        self._player_1 = player_1
        self._player_2 = player_2
        self._active_player = player_1
        self._inactive_player = player_2

        # The last 3 entries of the board state includes initiative (0 for
        # player 1, 1 for player 2) player 2 last move, and player 1 last move
        self._board_state = [Board.BLANK] * (width * height + 3)
        self._board_state[-1] = Board.NOT_MOVED
        self._board_state[-2] = Board.NOT_MOVED

    def hash(self):
        return str(self._board_state).__hash__()

    @property
    def active_player(self):
        """The object registered as the player holding initiative in the
        current game state.
        """
        return self._active_player

    @property
    def inactive_player(self):
        """The object registered as the player in waiting for the current
        game state.
        """
        return self._inactive_player

    def get_opponent(self, player):
        """Return the opponent of the supplied player.

        Parameters
        ----------
        player : object
            An object registered as a player in the current game. Raises an
            error if the supplied object is not registered as a player in
            this game.

        Returns
        -------
        object
            The opponent of the input player object.
        """
        if player == self._active_player:
            return self._inactive_player
        elif player == self._inactive_player:
            return self._active_player
        raise RuntimeError("`player` must be an object registered as a player in the current game.")

    def copy(self):
        """ Return a deep copy of the current board. """
        new_board = Board(self._player_1, self._player_2, width=self.width, height=self.height)
        new_board.move_count = self.move_count
        new_board._active_player = self._active_player
        new_board._inactive_player = self._inactive_player
        new_board._board_state = copy(self._board_state)
        return new_board

    def forecast_move(self, move):
        """Return a deep copy of the current game with an input move applied to
        advance the game one ply.

        Parameters
        ----------
        move : (int, int)
            A coordinate pair (row, column) indicating the next position for
            the active player on the board.

        Returns
        -------
        isolation.Board
            A deep copy of the board with the input move applied.
        """
        new_board = self.copy()
        new_board.apply_move(move)
        return new_board

    def move_is_legal(self, move):
        """Test whether a move is legal in the current game state.

        Parameters
        ----------
        move : (int, int)
            A coordinate pair (row, column) indicating the next position for
            the active player on the board.

        Returns
        -------
        bool
            Returns True if the move is legal, False otherwise
        """
        idx = move[0] + move[1] * self.height
        return (0 <= move[0] < self.height and 0 <= move[1] < self.width and
                self._board_state[idx] == Board.BLANK)

    def get_blank_spaces(self):
        """Return a list of the locations that are still available on the board.
        """
        return [(i, j) for j in range(self.width) for i in range(self.height)
                if self._board_state[i + j * self.height] == Board.BLANK]

    def get_player_location(self, player):
        """Find the current location of the specified player on the board.

        Parameters
        ----------
        player : object
            An object registered as a player in the current game.

        Returns
        -------
        (int, int) or None
            The coordinate pair (row, column) of the input player, or None
            if the player has not moved.
        """
        if player == self._player_1:
            if self._board_state[-1] == Board.NOT_MOVED:
                return Board.NOT_MOVED
            idx = self._board_state[-1]
        elif player == self._player_2:
            if self._board_state[-2] == Board.NOT_MOVED:
                return Board.NOT_MOVED
            idx = self._board_state[-2]
        else:
            raise RuntimeError(
                "Invalid player in get_player_location: {}".format(player))
        w = idx // self.height
        h = idx % self.height
        return (h, w)

    def get_legal_moves(self, player=None):
        """Return the list of all legal moves for the specified player.

        Parameters
        ----------
        player : object (optional)
            An object registered as a player in the current game. If None,
            return the legal moves for the active player on the board.

        Returns
        -------
        list<(int, int)>
            The list of coordinate pairs (row, column) of all legal moves
            for the player constrained by the current game state.
        """
        if player is None:
            player = self.active_player
        return self.__get_moves(self.get_player_location(player))

    def apply_move(self, move):
        """Move the active player to a specified location.

        Parameters
        ----------
        move : (int, int)
            A coordinate pair (row, column) indicating the next position for
            the active player on the board.
        """
        idx = move[0] + move[1] * self.height
        last_move_idx = int(self.active_player == self._player_2) + 1
        self._board_state[-last_move_idx] = idx
        self._board_state[idx] = 1
        self._board_state[-3] ^= 1
        self._active_player, self._inactive_player = self._inactive_player, self._active_player
        self.move_count += 1

    def is_winner(self, player):
        """ Test whether the specified player has won the game. """
        return player == self._inactive_player and not self.get_legal_moves(self._active_player)

    def is_loser(self, player):
        """ Test whether the specified player has lost the game. """
        return player == self._active_player and not self.get_legal_moves(self._active_player)

    def utility(self, player):
        """Returns the utility of the current game state from the perspective
        of the specified player.

                    /  +infinity,   "player" wins
        utility =  |   -infinity,   "player" loses
                    \          0,    otherwise

        Parameters
        ----------
        player : object (optional)
            An object registered as a player in the current game. If None,
            return the utility for the active player on the board.

        Returns
        ----------
        float
            The utility value of the current game state for the specified
            player. The game has a utility of +inf if the player has won,
            a value of -inf if the player has lost, and a value of 0
            otherwise.
        """
        if not self.get_legal_moves(self._active_player):

            if player == self._inactive_player:
                return float("inf")

            if player == self._active_player:
                return float("-inf")

        return 0.

    def __get_moves(self, loc):
        """Generate the list of possible moves for an L-shaped motion (like a
        knight in chess).
        """
        if loc == Board.NOT_MOVED:
            return self.get_blank_spaces()

        r, c = loc
        directions = [(-2, -1), (-2, 1), (-1, -2), (-1, 2),
                      (1, -2), (1, 2), (2, -1), (2, 1)]
        directions = [(-1, -1), (-1, 1), (-1, -1), (-1, 1),
                      (1, -1), (1, 1), (1, -1), (1, 1)]
        valid_moves = [(r + dr, c + dc) for dr, dc in directions
                       if self.move_is_legal((r + dr, c + dc))]
        random.shuffle(valid_moves)
        return valid_moves

    def print_board(self):
        """DEPRECATED - use Board.to_string()"""
        return self.to_string()

    def to_string(self, symbols=['1', '2']):
        """Generate a string representation of the current game state, marking
        the location of each player and indicating which cells have been
        blocked, and which remain open.
        """
        p1_loc = self._board_state[-1]
        p2_loc = self._board_state[-2]

        col_margin = len(str(self.height - 1)) + 1
        prefix = "{:<" + "{}".format(col_margin) + "}"
        offset = " " * (col_margin + 3)
        out = offset + '   '.join(map(str, range(self.width))) + '\n\r'
        for i in range(self.height):
            out += prefix.format(i) + ' | '
            for j in range(self.width):
                idx = i + j * self.height
                if not self._board_state[idx]:
                    out += ' '
                elif p1_loc == idx:
                    out += symbols[0]
                elif p2_loc == idx:
                    out += symbols[1]
                else:
                    out += '-'
                out += ' | '
            out += '\n\r'

        return out

    def play(self, time_limit=TIME_LIMIT_MILLIS):
        """Execute a match between the players by alternately soliciting them
        to select a move and applying it in the game.

        Parameters
        ----------
        time_limit : numeric (optional)
            The maximum number of milliseconds to allow before timeout
            during each turn.

        Returns
        ----------
        (player, list<[(int, int),]>, str)
            Return multiple including the winning player, the complete game
            move history, and a string indicating the reason for losing
            (e.g., timeout or invalid move).
        """
        move_history = []

        time_millis = lambda: 1000 * timeit.default_timer()

        while True:

            legal_player_moves = self.get_legal_moves()
            game_copy = self.copy()

            move_start = time_millis()
            time_left = lambda : time_limit - (time_millis() - move_start)
            curr_move = self._active_player.get_move(game_copy, time_left)
            move_end = time_left()

            if curr_move is None:
                curr_move = Board.NOT_MOVED

            if move_end < 0:
                return self._inactive_player, move_history, "timeout"

            if curr_move not in legal_player_moves:
                if len(legal_player_moves) > 0:
                    return self._inactive_player, move_history, "forfeit"
                return self._inactive_player, move_history, "illegal move"

            move_history.append(list(curr_move))

            self.apply_move(curr_move)


In [195]:
"""This file contains a collection of player classes for comparison with your
own agent and example heuristic functions.

    ************************************************************************
    ***********  YOU DO NOT NEED TO MODIFY ANYTHING IN THIS FILE  **********
    ************************************************************************
"""



def null_score(game, player):
    """This heuristic presumes no knowledge for non-terminal states, and
    returns the same uninformative value for all other states.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state.
    """

    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    return 0.


def open_move_score(game, player):
    """The basic evaluation function described in lecture that outputs a score
    equal to the number of moves open for your computer player on the board.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    return float(len(game.get_legal_moves(player)))


def improved_score(game, player):
    """The "Improved" evaluation function discussed in lecture that outputs a
    score equal to the difference in the number of moves available to the
    two players.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - opp_moves)


def center_score(game, player):
    """Outputs a score equal to square of the distance from the center of the
    board to the position of the player.

    This heuristic is only used by the autograder for testing.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    w, h = game.width, game.height
    y, x = game.get_player_location(player)
    return float((h - y)**2 + (w - x)**2)


class RandomPlayer():
    """Player that chooses a move randomly."""

    def get_move(self, game, time_left):
        """Randomly select a move from the available legal moves.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            A randomly selected legal move; may return (-1, -1) if there are
            no available legal moves.
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)
        return legal_moves[randint(0, len(legal_moves) - 1)]


class GreedyPlayer():
    """Player that chooses next move to maximize heuristic score. This is
    equivalent to a minimax search agent with a search depth of one.
    """

    def __init__(self, score_fn=open_move_score):
        self.score = score_fn

    def get_move(self, game, time_left):
        """Select the move from the available legal moves with the highest
        heuristic score.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            The move in the legal moves list with the highest heuristic score
            for the current game state; may return (-1, -1) if there are no
            legal moves.
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)
        _, move = max([(self.score(game.forecast_move(m), self), m) for m in legal_moves])
        return move


class HumanPlayer():
    """Player that chooses a move according to user's input."""

    def get_move(self, game, time_left):
        """
        Select a move from the available legal moves based on user input at the
        terminal.

        **********************************************************************
        NOTE: If testing with this player, remember to disable move timeout in
              the call to `Board.play()`.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            The move in the legal moves list selected by the user through the
            terminal prompt; automatically return (-1, -1) if there are no
            legal moves
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)

        print(game.to_string()) #display the board for the human player
        print(('\t'.join(['[%d] %s' % (i, str(move)) for i, move in enumerate(legal_moves)])))

        valid_choice = False
        while not valid_choice:
            try:
                index = int(input('Select move index:'))
                valid_choice = 0 <= index < len(legal_moves)

                if not valid_choice:
                    print('Illegal move! Try again.')

            except ValueError:
                print('Invalid index! Try again.')

        return legal_moves[index]


if __name__ == "__main__":

    # create an isolation board (by default 7x7)
    player1 = RandomPlayer()
    player2 = GreedyPlayer()
    game = Board(player1, player2)

    # place player 1 on the board at row 2, column 3, then place player 2 on
    # the board at row 0, column 5; display the resulting board state.  Note
    # that the .apply_move() method changes the calling object in-place.
    game.apply_move((2, 3))
    game.apply_move((0, 5))
    print(game.to_string())

    # players take turns moving on the board, so player1 should be next to move
    assert(player1 == game.active_player)

    # get a list of the legal moves available to the active player
    print(game.get_legal_moves())

    # get a successor of the current state by making a copy of the board and
    # applying a move. Notice that this does NOT change the calling object
    # (unlike .apply_move()).
    new_game = game.forecast_move((1, 1))
    assert(new_game.to_string() != game.to_string())
    print("\nOld state:\n{}".format(game.to_string()))
    print("\nNew state:\n{}".format(new_game.to_string()))

    # play the remainder of the game automatically -- outcome can be "illegal
    # move", "timeout", or "forfeit"
    winner, history, outcome = game.play()
    print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
    print(game.to_string())
    print("Move history:\n{!s}".format(history))


     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   |   |   |   |   |   |   | 
2  |   |   |   | 1 |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 

[(3, 4), (1, 2), (3, 2), (3, 2), (1, 4), (3, 4), (1, 4), (1, 2)]

Old state:
     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   |   |   |   |   |   |   | 
2  |   |   |   | 1 |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 


New state:
     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   | 1 |   |   |   |   |   | 
2  |   |   |   | - |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 


Winner: <__main__.GreedyPlayer object at 0x108851e80>
Outcome: illegal move
     0   1   2   3   4  

In [426]:
def iterative_deepening_depth_limited(game, depth):
    """
    Find the scores for all the possible moves down to the given depth
    
    Parameters
    ----------
    game : isolation.Board
        An instance of the Isolation game `Board` class representing the
        current game state

    depth : int
        Depth is an integer representing the maximum number of plies to
        search in the game tree before aborting

    Returns
    -------
    scores: dictionary
        scores for all the possible board states down to the depth
    """
    
    
    # Used for BFS search for each level
    queue = [['0',board]]
    
    # For capturing the scores at the bottom level
    scores_dict = dict()
    scores_dict['0'] = [float('-inf'),]

    while queue and depth:
        
        # BFS so first in - first out and therefore using the first board state
        l, g = queue.pop(0)
        
        # Finding the level explored so far based on the number of blank spaces
        blank_spaces = g.get_blank_spaces()
        level = (board.height * board.width - len(blank_spaces))

        # Assuming we are maximizing the chance of player1, then the top level
        # will do maximizing and then alternate after
        player = [player1,player2][(level)%2]
        
        # Possible moves are basically overlap of blank spaces and the possible moved
        possible_moves = g.get_legal_moves()
        possible_moves = list(set(possible_moves) & set(blank_spaces))


        # If reached the bottom level, just return the scores -- Nothing else to do
        if level == depth:
            for i in range(len(possible_moves)):
                #for move in possible_moves:
                new_board = g.forecast_move(possible_moves[i])
                scores_dict[str(l) + '-' + str(i+1)] = [improved_score(new_board,player),possible_moves[i]]
                return scores_dict
        else:
            for i in range(len(possible_moves)):
                #for move in possible_moves:
                new_board = g.forecast_move(possible_moves[i])
                scores_dict[str(l) + '-' + str(i+1)] = [improved_score(new_board,player),possible_moves[i]]
                queue.append([str(l) + '-' + str(i+1),new_board])

            
    return scores_dict


depth = 2
width = 3
height = 3

player1 = GreedyPlayer()
player2 = GreedyPlayer()
board = Board(player_1=player1, player_2=player2, width=width, height=height)

scores = iterative_deepening_depth_limited(board, depth)
len(scores)

83

Finding the last level and dropping the rest

In [400]:
level = depth - 1

while level >= 0:
        
    # Find all the nodes in that level
    #nodes = [x for x in scores if len(x) == (2*level+1)]
    nodes = [x for x in scores if (len(x.split('-')) - 1) == level]
    
    # Find Min/Max for each node and keep track of all the moves that led to that
    
    if level %2 == 0:

        for node in nodes:
            
            max_score = float('-inf')
            next_move = ('no','no')
            scores[node].append(next_move)

            # The branching factor is definitely less than the board size!
            try:
                # Go over all childs
                for i in range(1, width*height+1):
                    
                    # Child Node name
                    child_node = node + '-' + str(i)
                    child_score = scores[child_node][0]
                    if child_score > max_score:
                        max_score = max(child_score,max_score)
                        # Updated the score with the new max
                        scores[node][0] = max_score # the same as child_score
                        # Keep track of the moves
                        scores[node][-1] = scores[child_node][1:]
            except:
                continue
                

    
    if level %2 == 1:
        
        for node in nodes:
            
            min_score = float('inf')
            next_move = ('no','no')
            scores[node].append(next_move)

            # The branching factor is definitely less than the board size!
            try:
                # Go over all childs
                for i in range(1, width*height+1):
                    # Child Node name
                    child_node = node + '-' + str(i)
                    child_score = scores[child_node][0]
                    if child_score < min_score:
                        min_score = min(child_score,max_score)
                        # Updated the score with the new max
                        scores[node][0] = min_score # the same as child_score
                        # Keep track of the moves
                        scores[node][-1] = scores[child_node][1:]
            except:
                continue
        
    level -= 1
    



MINIMIZE


In [401]:
scores

{'0': [-2.0, [(0, 0), [(1, 2), [(1, 1)]]]],
 '0-1': [-4.0, (0, 1), [(1, 2), [(1, 0)]]],
 '0-1-1': [-4.0, (1, 2), [(1, 0)]],
 '0-1-1-1': [-4.0, (1, 0)],
 '0-1-10': [2.0, (0, 2), [(1, 2)]],
 '0-1-10-1': [2.0, (1, 2)],
 '0-1-10-2': [-2.0, (1, 0)],
 '0-1-11': [inf, (0, 3), [(1, 2)]],
 '0-1-11-1': [inf, (1, 2)],
 '0-1-11-2': [0.0, (1, 0)],
 '0-1-2': [4.0, (0, 0), [(1, 2)]],
 '0-1-2-1': [4.0, (1, 2)],
 '0-1-2-2': [0.0, (1, 0)],
 '0-1-3': [2.0, (1, 3), [(1, 2)]],
 '0-1-3-1': [2.0, (1, 2)],
 '0-1-3-2': [-2.0, (1, 0)],
 '0-1-4': [2.0, (2, 1), [(1, 2)]],
 '0-1-4-1': [2.0, (1, 2)],
 '0-1-4-2': [-2.0, (1, 0)],
 '0-1-5': [4.0, (2, 0), [(1, 2)]],
 '0-1-5-1': [4.0, (1, 2)],
 '0-1-5-2': [0.0, (1, 0)],
 '0-1-6': [-2.0, (1, 1), [(1, 2)]],
 '0-1-6-1': [-2.0, (1, 2)],
 '0-1-6-2': [-6.0, (1, 0)],
 '0-1-7': [inf, (2, 3), [(1, 2)]],
 '0-1-7-1': [inf, (1, 2)],
 '0-1-7-2': [0.0, (1, 0)],
 '0-1-8': [2.0, (2, 2), [(1, 2)]],
 '0-1-8-1': [2.0, (1, 2)],
 '0-1-8-2': [-2.0, (1, 0)],
 '0-1-9': [4.0, (1, 0), [(1, 2)]],